In [1]:
import pytorchfi
import torch
import torchvision
import torchvision.models as models
import random
import copy
import numpy as np

from pytorchfi.core import fault_injection
from pytorchfi.neuron_error_models import single_bit_flip_func
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

## Settings

In [2]:
model = models.resnet18(pretrained=True)

seed = 1234

batch_size = 50
img_size = 224
channels = 3

use_gpu = torch.cuda.is_available()
quant_bits = 8
layer_type = [torch.nn.Conv2d]

imagenet_dataset_dir = 'D:\dataset'

print(use_gpu)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Q/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 45.2MB/s]

True


In [3]:
random.seed(seed)
torch.manual_seed(seed)

## Class Original_Model
원본 모델에 forward hook을 등록해서 quantization에 필요한 max value를 얻을 수 있도록 함

In [4]:
class Original_Model(torch.nn.Module):
    def __init__(self, model, layer_type, layer_num, *args):
        super().__init__(*args)
        self.model = model
        self.max_value = None
        self.layer_type = layer_type
        self.layer_num = layer_num
        self.fhooks = []

        self.layer_cnt = 0
        self._register_hook_to_target_layer()

    def forward(self, x):
        self.max_value = None
        out = self.model(x)
        return out, self.max_value.item()
    
    def forward_hook(self):
        def hook(module, input, output):
            self.max_value = torch.max(output)
        return hook

    def _register_hook_to_target_layer(self):
        
        layer_idx = -1

        for name, module in self.model.named_modules():
            if len(list(module.children())) == 0 and type(module) in self.layer_type:
                layer_idx += 1

                if layer_idx == self.layer_num:
                    self.fhooks.append(module.register_forward_hook(self.forward_hook()))
                    print(f'Registered forward hook to {module}')

        self.layer_cnt = layer_idx + 1

## Class Fault_Injection_Experiment

In [5]:
class Fault_Injection_Experiment:
    def __init__(self, model, batch_size, channels, img_size, use_gpu=False, quant_bits=8, seed=1234):
        if use_gpu:
            model.to(device='cuda')

        self.original_model = copy.deepcopy(model)
        self.error_model = copy.deepcopy(model)

        self.batch_size = batch_size
        self.channels = channels
        self.img_size = img_size
        self.use_gpu = use_gpu
        self.quant_bits = quant_bits

        self.seed = seed

    def set_models(self, layer_type, layer_num):
        self.layer_type = layer_type
        self.layer_num = layer_num
        
        self.original_model = Original_Model(self.original_model, layer_type, layer_num)
        if self.use_gpu:
            self.original_model.cuda()
        
        self.error_model = single_bit_flip_func(
            model = self.error_model, 
            batch_size = self.batch_size, 
            input_shape = [self.channels, self.img_size, self.img_size], 
            use_gpu = self.use_gpu,
            bits = self.quant_bits,
            layer_types = layer_type
        )

    def run(self, image):
        if self.use_gpu:
            image = image.to(device='cuda')
        
        # 원본에 inference 진행
        self.original_model.eval()
        with torch.no_grad():
            original_output, max_value = self.original_model(image)

        # injection 위치 지정
        random.seed(self.seed)
        layer_nums = []
        dim1 = []
        dim2 = []
        dim3 = []

        for _ in range(self.batch_size):
            layer, C, H, W = pytorchfi.neuron_error_models.random_neuron_location(self.error_model, layer=self.layer_num)
            layer_nums.append(layer)
            dim1.append(C)
            dim2.append(H)
            dim3.append(W)

        # error model 만들기
        self.error_model.set_conv_max([max_value for _ in range(self.error_model.get_total_layers())])

        error_model = self.error_model.declare_neuron_fi(
            batch = [i for i in range(self.batch_size)],
            layer_num = layer_nums,
            dim1 = dim1,
            dim2 = dim2,
            dim3 = dim3,
            function = self.error_model.single_bit_flip_signed_across_batch
        )        

        # error model에 inference 진행
        error_model.eval()
        with torch.no_grad():
            error_output = error_model(image)

        return original_output, error_output


## Data Preprocessing

In [6]:
# ILSVRC2012 validation set (https://image-net.org/challenges/LSVRC/2012/2012-downloads.php)
# normallization info from https://pytorch.org/vision/main/models/generated/torchvision.models.alexnet.html
preprocess = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

imagenet_dataset = torchvision.datasets.ImageNet(
    root = imagenet_dataset_dir,
    split = 'val',
    transform = preprocess
)

dataset = torch.utils.data.DataLoader(imagenet_dataset, batch_size = batch_size, shuffle = True)

## Main

In [7]:
# single bit flip을 일으킬 모델 만들기
if use_gpu:
    model.to(device='cuda')

tmp = single_bit_flip_func(
    model = copy.deepcopy(model),
    batch_size = batch_size, 
    input_shape = [channels, img_size, img_size], 
    use_gpu = use_gpu,
    bits = quant_bits,
    layer_types = layer_type
)

print(tmp.print_pytorchfi_layer_summary())

c:\Program Files\Python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


============================ PYTORCHFI INIT SUMMARY ==============================

Layer types allowing injections:
----------------------------------------------------------------------------------
   - Conv2d

Model Info:
----------------------------------------------------------------------------------
   - Shape of input into the model: (3 224 224 )
   - Batch Size: 50
   - CUDA Enabled: True

Layer Info:
----------------------------------------------------------------------------------
Layer #       Layer type  Dimensions         Weight Shape         Output Shape
----------------------------------------------------------------------------------
    0           Conv2d           4        [64, 3, 7, 7]    [1, 64, 112, 112]
    1           Conv2d           4       [64, 64, 3, 3]      [1, 64, 56, 56]
    2           Conv2d           4       [64, 64, 3, 3]      [1, 64, 56, 56]
    3           Conv2d           4       [64, 64, 3, 3]      [1, 64, 56, 56]
    4           Conv2d           

In [8]:
results = []

for layer_num in range(tmp.get_total_layers()):

    # fi
    test = Fault_Injection_Experiment(
        model=model, 
        batch_size=batch_size,
        channels=channels, 
        img_size=img_size, 
        use_gpu=use_gpu, 
        quant_bits=quant_bits,
        seed=seed
    )

    test.set_models(layer_type=layer_type, layer_num = layer_num)

    # run    
    orig_correct_cnt = 0
    orig_error_diff_cnt = 0

    for images, labels in tqdm(dataset):

        orig_output, error_output = test.run(images)

        orig_output = torch.argmax(orig_output, dim=1).cpu().numpy()
        error_output = torch.argmax(error_output, dim=1).cpu().numpy()
        labels = labels.numpy()

        for i in range(len(labels)):
            # 원본 모델이 정답을 맞춘 경우
            if labels[i] == orig_output[i]:
                orig_correct_cnt += 1
                # 원본 모델이 정답을 맞췄는데 망가진 모델은 틀린 경우
                if orig_output[i] != error_output[i]:
                    orig_error_diff_cnt += 1

    # save result
    result = f'Layer #{layer_num}: {orig_error_diff_cnt} / {orig_correct_cnt} = {orig_error_diff_cnt / orig_correct_cnt * 100}%'
    print(result, end='\n\n')
    results.append(result)

Registered forward hook to Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


100%|██████████| 1000/1000 [08:10<00:00,  2.04it/s]


Layer #0: 2606 / 34880 = 7.471330275229358%

Registered forward hook to Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 1000/1000 [07:55<00:00,  2.10it/s]


Layer #1: 2559 / 34880 = 7.33658256880734%

Registered forward hook to Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


...

Layer #17: 2511 / 34880 = 7.198967889908257%

Registered forward hook to Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 1000/1000 [07:22<00:00,  2.26it/s]


Layer #18: 2542 / 34880 = 7.287844036697249%

Registered forward hook to Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 1000/1000 [07:23<00:00,  2.25it/s]

Layer #19: 2532 / 34880 = 7.259174311926605%



## Save Result

In [9]:
save_dir = 'resnet18_conv2d.txt'
f = open(save_dir, 'w')

f.write(tmp.print_pytorchfi_layer_summary())
f.write(f'\n\n===== Result =====\nQuantization bits: {quant_bits}\n')
for result in results:
    f.write(result + '\n')

f.close()

============================ PYTORCHFI INIT SUMMARY ==============================

Layer types allowing injections:
----------------------------------------------------------------------------------
   - Conv2d

Model Info:
----------------------------------------------------------------------------------
   - Shape of input into the model: (3 224 224 )
   - Batch Size: 50
   - CUDA Enabled: True

Layer Info:
----------------------------------------------------------------------------------
Layer #       Layer type  Dimensions         Weight Shape         Output Shape
----------------------------------------------------------------------------------
    0           Conv2d           4        [64, 3, 7, 7]    [1, 64, 112, 112]
    1           Conv2d           4       [64, 64, 3, 3]      [1, 64, 56, 56]
    2           Conv2d           4       [64, 64, 3, 3]      [1, 64, 56, 56]
    3           Conv2d           4       [64, 64, 3, 3]      [1, 64, 56, 56]
    4           Conv2d           